### Install the required packages

In [ ]:
#%pip install python-dotenv
#%pip install requests

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values,load_dotenv
import sys
sys.path.append('../common')
from helper import Helper
load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")

# Create a helper object
helper = Helper(environment, "genai/azure/reporting", api_key, region_override)

### GET Model and UPDATE Replacement Suggestions

In [ ]:
# The PUT endpoint updates and overrides the model replacements file each time.
# We don't want our regression to overwrite what is in blob storage,
# so we first make a GET request to get the current model replacements file if it exists 
# otherwise we use some sample data.

sample_model_replacements = {
    "value": [
        {
            "modelName": "dall-e-2",
            "version": "2",
            "suggestedReplacements": {
                "value": [
                   {
                       "modelName": "dalle-3",
                       "version": ""
                   }
                ]
             }
        },
        {
             "modelName": "gpt-35-turbo",
             "version": "0613",
             "suggestedReplacements": {
                "value": [
                    {
                        "modelName": "gpt-35-turbo",
                        "version": "gpt-4o-mini"
                    },
                    {
                        "modelName": "gpt-4o-mini",
                        "version": ""
                    }
                ]
             }
        }  
    ]
}

# Make the first GET request
get_model_replacements = helper.get("/openai/model-replacements", None)

# If the GET request fails with a 404, we use the sample data. The GET request fails if the model replacements file does not exist (meaning no PUT request has been made yet)
model_replacements = sample_model_replacements if get_model_replacements.status_code != 200 else get_model_replacements.json

# We then make a PUT request to update the model replacements file with the same data
response = helper.put("/openai/model-replacements", model_replacements)

# Validate the response code
assert response.status_code == 201

# Assert that the response body is the same as the model replacements file
assert (response.json == model_replacements)

# At this point, we have successfully updated the model replacements file with the sample data if it was not already populated by the model librarian.
# So we can make a GET request to get the model replacements file and validate that it is not empty.
get_model_replacements = helper.get("/openai/model-replacements", None)

# Validate the response code
assert get_model_replacements.status_code == 200

# Get the model replacements from the response
model_replacements = get_model_replacements.json

# Validate the response has content
assert model_replacements is not None

# Output the response for troubleshooting purposes
print("GET Model Replacements Response: " + str(model_replacements))

# Access the suggestedReplacements property of the list
suggestions_list = helper.get_property_values(model_replacements, 'suggestedReplacements')

# Validate the suggested replacements is not empty
assert len(suggestions_list) > 0


### PUT Gather Model Deployments Information

In [ ]:
# Make a PUT request
response = helper.put("/openai/deployments", None)

# Validate the response code
assert response.status_code == 200, "Response status code is not 200"

# Parse the response JSON
response_data = response.json if isinstance(response.json, dict) else response.json()

# Assert that the response body is not empty
assert response_data, "Response body is empty"

# Assert that the response contains the 'value' field
assert 'value' in response_data, "Response does not contain 'value' field"

# Assert that the 'value' field is an empty array
assert isinstance(response_data['value'], list), "'value' is not a list"
assert response_data['value'], "'value' is a empty array"


### POST Generate CSV report

In [ ]:
# Make a POST request to Generate CSV report
response = helper.post("/openai/models-report", None)

# Validate the response code
assert response.status_code == 200, "response status code is not 200"

# Parse the response JSON
response_data = response.json if isinstance(response.json, dict) else response.json()

# Assert that the response body is not empty
assert response_data, "Response body is empty"

# Assert that the response contains the field 'message' with the expected value
assert 'message' in response_data, "Response does not contain 'message' field"

# Assert that the response message is correct
assert response.json['message'] == 'CSV file generated and stored successfully.', \
    f"Unexpected message value: {response.json['message']}"
